### [Serializing LLM Pipelines](https://haystack.deepset.ai/tutorials/29_serializing_pipelines)

In [ ]:
#! Solo es para haystack sepa que tutorial se esta ejecutando
from haystack.telemetry import tutorial_running
tutorial_running(29)

### 1. Creando una tubería simple

In [1]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator

template = """
Please create a summary about the following topic:
{{ topic }}
"""

builder = PromptBuilder(template=template)
llm = HuggingFaceLocalGenerator(
    model="google/flan-t5-large", task="text2text-generation", generation_kwargs={"max_new_tokens": 150}
)

pipeline = Pipeline()
pipeline.add_component(name="builder", instance=builder)
pipeline.add_component(name="llm", instance=llm)

pipeline.connect("builder", "llm")

🚅 Components
  - builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - builder.prompt -> llm.prompt (str)

In [5]:
topic = "Climate change"
result = pipeline.run(data={"builder": {"topic": topic}})
print(result["llm"]["replies"][0])

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


Climate change is a major threat to the planet.


### 2. Serializar el pipeline a YAML

In [6]:
yaml_pipeline = pipeline.dumps()

print(yaml_pipeline)

components:
  builder:
    init_parameters:
      required_variables: null
      template: '

        Please create a summary about the following topic:

        {{ topic }}

        '
      variables: null
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cuda:0
        model: google/flan-t5-large
        task: text2text-generation
      stop_words: null
      streaming_callback: null
      token:
        env_vars:
        - HF_API_TOKEN
        - HF_TOKEN
        strict: false
        type: env_var
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}



### 3. Edición de un pipeline 

In [7]:
yaml_pipeline = """
components:
  builder:
    init_parameters:
      template: "\nPlease translate the following to French: \n{{ sentence }}\n"
    type: haystack.components.builders.prompt_builder.PromptBuilder
  llm:
    init_parameters:
      generation_kwargs:
        max_new_tokens: 150
      huggingface_pipeline_kwargs:
        device: cpu
        model: google/flan-t5-large
        task: text2text-generation
        token: null
      stop_words: null
    type: haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator
connections:
- receiver: llm.prompt
  sender: builder.prompt
max_runs_per_component: 100
metadata: {}
"""

### 4. Deserializando una canalización de YAML de vuelta a python

In [12]:
from haystack import Pipeline

new_pipeline = Pipeline.loads(yaml_pipeline)
new_pipeline

🚅 Components
  - builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - builder.prompt -> llm.prompt (str)

In [13]:
new_pipeline.run(data={"builder": {"sentence": "I love capybaras"}})

Device set to use cpu


{'llm': {'replies': ['Je me félicite des capybaras !']}}